### Importing libraries and toolbox

In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import show

#### All `sompy.` terms refer to the toolbox files

# from sompy.sompy_isom import SOMFactory as smf   # (In case of iSOM)
from sompy.sompy_csom import SOMFactory as smf  # (In case of cSOM)
    
from sompy.visualization.viz_functions import Visualization_func
from sompy.visualization.comp_planes import som_cplanes
from sompy.visualization.hitmaps import som_hitmap
from sompy.aux_fun import aux_fun

### Loading input data

1. `n_array`: Input data which is a numpy array
2. `comp_names`: Component names i.e. names of each design variable/feature

In [2]:
file = open('spherical_data.csv')
n_array = np.genfromtxt(file, delimiter=',',skip_header=1)
data = pd.read_csv('spherical_data.csv')
file.close()
comp_names = list(data.columns)
comp_names

['x', 'y', 'z']

In [3]:
n_array

array([[ 0.58137 , -0.81581 ,  1.0035  ],
       [ 0.97213 , -0.063283,  0.94904 ],
       [ 1.9926  , -1.8059  ,  7.2318  ],
       [-1.2739  , -2.2039  ,  6.4801  ],
       [-4.2479  , -3.527   , 30.484   ],
       [ 0.17807 ,  4.4772  , 20.077   ],
       [-4.5997  ,  3.3209  , 32.186   ],
       [ 2.5866  , -0.50954 ,  6.9503  ],
       [-4.6566  ,  0.38376 , 21.832   ],
       [ 4.8635  ,  1.707   , 26.568   ],
       [ 2.8898  ,  1.2355  ,  9.8773  ],
       [ 4.3859  , -2.7202  , 26.635   ],
       [ 0.48171 ,  4.9728  , 24.96    ],
       [-3.082   , -3.7763  , 23.759   ],
       [-2.7131  ,  3.5447  , 19.926   ],
       [ 3.4293  , -4.6993  , 33.844   ],
       [ 4.9878  , -1.3169  , 26.612   ],
       [-1.2819  ,  2.3422  ,  7.1294  ],
       [ 3.1516  ,  2.2175  , 14.85    ],
       [-3.8262  , -3.9105  , 29.931   ],
       [ 1.0564  ,  4.8625  , 24.759   ],
       [-0.541   ,  1.3307  ,  2.0635  ],
       [ 2.4692  , -2.9433  , 14.76    ],
       [-2.3659  ,  1.9103  ,  9.2

### Build initial SOM structure

Following attributes are set:
1. `sm._normalizer` = Instance of class `VarianceNormalizer` or `RangeNormalizer`. Default is `RangeNormalizer`. (See `normalization.py`)
2. `sm._data`: This is the normalized data in case `normalization` parameter is provided.
3. `sm._dim`: Number of features (3 for this example)
4. `sm._dlen`: Number of data points (40 for this example)
5. `sm.lattice`: Lattice type `hexa`. Only `hexa` allowed.
6. `sm.data_raw`: Original data without normalization.
7. `sm.neighborhood`: Only "gaussian" 
8. `sm.training`: Only "batch"
9. `sm.codebook`: Instance of class `Codebook`. (See `codebook.py`)
10. `sm._distance_matrix`: Distance matrix (size: m x m) which has distance between each nodes. 'm' is number of map nodes

In [4]:
sm = smf.build(n_array,normalization='range',initialization='pca',mapshape="planar",mapsize = [6,6],lattice="hexa",neighborhood='gaussian',
          training='batch',component_names=comp_names)

In [5]:
sm.codebook.matrix

array([6, 6])

### Initialization phase

Two types:
1. `sm.som_randinit()`: For random initialization (only for cSOM)
2. `sm.som_lininit()`: For PCA initialization (for both cSOM and iSOM)

At the end of this execution, following attributes will be updated:
1. `sm.codebook.matrix`: This is the initialized matrix
2. `sm.initialized_matrix`: This is also the initialized matrix

In [6]:
sm.som_lininit()

 pca_linear_initialization took: 0.012000 seconds
 som_lininit took: 0.013000 seconds


In [7]:
sm.codebook.matrix

array([[ 5.985500e-02, -5.406920e-01,  7.421120e-01],
       [ 2.478850e-01, -2.785010e-01,  5.686790e-01],
       [ 4.535440e-01, -2.698740e-01,  2.656810e-01],
       [ 6.415740e-01, -7.683000e-03,  9.224800e-02],
       [ 8.472340e-01,  9.450000e-04, -2.107510e-01],
       [ 1.035264e+00,  2.631360e-01, -3.841830e-01],
       [ 4.222500e-02, -2.871290e-01,  8.716770e-01],
       [ 2.302550e-01, -2.493800e-02,  6.982440e-01],
       [ 4.359140e-01, -1.631000e-02,  3.952460e-01],
       [ 6.239440e-01,  2.458800e-01,  2.218130e-01],
       [ 8.296040e-01,  2.545080e-01, -8.118500e-02],
       [ 1.017634e+00,  5.166990e-01, -2.546180e-01],
       [ 2.459500e-02, -3.356600e-02,  1.001242e+00],
       [ 2.126250e-01,  2.286250e-01,  8.278090e-01],
       [ 4.182850e-01,  2.372530e-01,  5.248110e-01],
       [ 6.063140e-01,  4.994430e-01,  3.513780e-01],
       [ 8.119740e-01,  5.080710e-01,  4.838000e-02],
       [ 1.000004e+00,  7.702620e-01, -1.250530e-01],
       [ 6.965000e-03,  2.19

### SOM Training phase

Following parameters are fixed here: `njob`=1, `shared_memory`=False, `train_len_factor`=1
User defined parameters:
1. `verbose`: Can be `info` or `None`
2. `train_rough_len`: Rough training length (See function `rough_train()` in sompy_csom.py or sompy_isom.py)
3. `train_finetune_len`: Fine training length (See function `finetune_train()` in sompy_csom.py or sompy_isom.py)

At the end of training, following attributes will be updated:
1. `sm._bmu`: A numpy array of size (2 x dlen); First row are indices of BMU nodes, Second row are distance values to BMU nodes
2. `sm._codebook.matrix`: Trained weights of SOM after training

In [8]:
sm.train(n_job = 1, shared_memory = False, verbose='info',train_rough_len=200, train_finetune_len=200,
                  train_len_factor=1) # To give train parameters and to train the SOM

 Training...
 Rough training...
 radius_ini: 1.000000 , radius_final: 1.000000, trainlen: 200

 epoch: 1 ---> elapsed time:  0.003000, quantization error: 0.276450

 epoch: 2 ---> elapsed time:  0.002000, quantization error: 0.243646

 epoch: 3 ---> elapsed time:  0.001000, quantization error: 0.182852

 epoch: 4 ---> elapsed time:  0.003000, quantization error: 0.170087

 epoch: 5 ---> elapsed time:  0.002000, quantization error: 0.168744

 epoch: 6 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 7 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 8 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 9 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 10 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 11 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 12 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 13 ---> elapsed time:  0.002000, quantization error: 0

 epoch: 115 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 116 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 117 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 118 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 119 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 120 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 121 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 122 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 123 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 124 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 125 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 126 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 127 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 128 ---> elapsed time:  0.002000, quantization error: 0.

 epoch: 29 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 30 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 31 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 32 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 33 ---> elapsed time:  0.003000, quantization error: 0.166416

 epoch: 34 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 35 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 36 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 37 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 38 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 39 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 40 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 41 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 42 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch

 epoch: 144 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 145 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 146 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 147 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 148 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 149 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 150 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 151 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 152 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 153 ---> elapsed time:  0.002000, quantization error: 0.166416

 epoch: 154 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 155 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 156 ---> elapsed time:  0.001000, quantization error: 0.166416

 epoch: 157 ---> elapsed time:  0.001000, quantization error: 0.

In [9]:
sm._bmu[0]

array([ 5.,  5., 10.,  4., 12., 33., 30., 17., 24., 35., 17., 14., 33.,
        0., 31., 14., 15., 26., 23., 12., 33.,  5.,  9., 26., 30., 27.,
       11., 19.,  4., 14., 35.,  1.,  4., 15., 34., 12., 35.,  2.,  2.,
       24.])

In [10]:
sm._bmu[1]

array([0.08614421, 0.11361515, 0.11819324, 0.16487765, 0.19747364,
       0.0768762 , 0.22321708, 0.11613543, 0.12375459, 0.24487007,
       0.1110581 , 0.29268243, 0.16539574, 0.08438559, 0.100374  ,
       0.42837519, 0.33080792, 0.30442835, 0.0942269 , 0.15965454,
       0.15663931, 0.22655184, 0.10457901, 0.26968337, 0.27235196,
       0.13493875, 0.14234208, 0.26942644, 0.08138567, 0.19099592,
       0.09014141, 0.08050452, 0.12147761, 0.18301319, 0.06539047,
       0.196127  , 0.04798728, 0.19851711, 0.13839564, 0.14963074])

In [11]:
sm.codebook.matrix

array([[0.267903, 0.112618, 0.646519],
       [0.377167, 0.133595, 0.603244],
       [0.410395, 0.201977, 0.407743],
       [0.536232, 0.282099, 0.298166],
       [0.504288, 0.366962, 0.124338],
       [0.549174, 0.432557, 0.084347],
       [0.27623 , 0.101399, 0.762922],
       [0.460092, 0.136699, 0.73228 ],
       [0.568777, 0.182663, 0.558875],
       [0.700908, 0.272461, 0.471958],
       [0.603437, 0.375441, 0.201951],
       [0.643624, 0.461818, 0.153664],
       [0.260444, 0.146021, 0.814099],
       [0.432773, 0.248793, 0.711535],
       [0.67272 , 0.216472, 0.661969],
       [0.75736 , 0.340158, 0.556278],
       [0.733988, 0.399461, 0.345742],
       [0.747175, 0.51857 , 0.269636],
       [0.191984, 0.349922, 0.72945 ],
       [0.286368, 0.508512, 0.585786],
       [0.575023, 0.393143, 0.582694],
       [0.626224, 0.574241, 0.498014],
       [0.765724, 0.508974, 0.430939],
       [0.780897, 0.636315, 0.39993 ],
       [0.120772, 0.610346, 0.690912],
       [0.217124, 0.71019

### Visualization 

Creating a visualization instance `vis` with attributes of sm.

In [12]:
vis = Visualization_func(sm)

Plotting component planes of codebook matrix (See `viz_functions.py` and `comp_planes.py`)

In [13]:
vis.plot_cplanes()

GridPlot(id='p1127', ...)

`aux_fun()`: Auxiliary functions: To find hits, data mapped to a given node, neighborhood nodes etc.

In [14]:
axf = aux_fun()
hits = axf.som_hits(sm, sm._data)  # sm._data here is normalized data

In [15]:
hits

array([1, 1, 2, 0, 3, 3, 0, 0, 0, 1, 1, 1, 3, 0, 3, 2, 0, 2, 0, 1, 0, 0,
       0, 1, 2, 0, 2, 1, 0, 0, 2, 1, 0, 3, 1, 3])

In [16]:
# Hit maps plotting
vis.plot_hitmap(hits, comp='all' , clr='red')   # Here, 'comp' takes index of last column of `n_array` data by default

Loading BokehJS ...

In [17]:
# from copy import deepcopy
# som_4umatrix = deepcopy(sm)
# som_4umatrix.set_codebook_matrix(vis.codebook)

In [18]:
# UMatrix plotting
U = axf.som_umat(sm, comp="all") 
vis.plot_umat(U)

Loading BokehJS ...

In [21]:
from copy import deepcopy
som_4umatrix = deepcopy(sm)
som_4umatrix.set_codebook_matrix(vis.codebook)

In [22]:
U = axf.som_umat(som_4umatrix, comp="all") 
vis.plot_umat(U)

Loading BokehJS ...

In [23]:
sm.codebook.matrix

array([[0.267903, 0.112618, 0.646519],
       [0.377167, 0.133595, 0.603244],
       [0.410395, 0.201977, 0.407743],
       [0.536232, 0.282099, 0.298166],
       [0.504288, 0.366962, 0.124338],
       [0.549174, 0.432557, 0.084347],
       [0.27623 , 0.101399, 0.762922],
       [0.460092, 0.136699, 0.73228 ],
       [0.568777, 0.182663, 0.558875],
       [0.700908, 0.272461, 0.471958],
       [0.603437, 0.375441, 0.201951],
       [0.643624, 0.461818, 0.153664],
       [0.260444, 0.146021, 0.814099],
       [0.432773, 0.248793, 0.711535],
       [0.67272 , 0.216472, 0.661969],
       [0.75736 , 0.340158, 0.556278],
       [0.733988, 0.399461, 0.345742],
       [0.747175, 0.51857 , 0.269636],
       [0.191984, 0.349922, 0.72945 ],
       [0.286368, 0.508512, 0.585786],
       [0.575023, 0.393143, 0.582694],
       [0.626224, 0.574241, 0.498014],
       [0.765724, 0.508974, 0.430939],
       [0.780897, 0.636315, 0.39993 ],
       [0.120772, 0.610346, 0.690912],
       [0.217124, 0.71019

In [24]:
vis.codebook

array([[-2.37643693e+00, -3.76409696e+00,  2.22162566e+01],
       [-1.27733943e+00, -3.55756371e+00,  2.07927273e+01],
       [-9.43095655e-01, -2.88429505e+00,  1.43617297e+01],
       [ 3.22711311e-01, -2.09543788e+00,  1.07571986e+01],
       [ 1.38342080e-03, -1.25990224e+00,  5.03913354e+00],
       [ 4.52896183e-01, -6.14073545e-01,  3.72363119e+00],
       [-2.29267481e+00, -3.87455587e+00,  2.60453287e+01],
       [-4.43188563e-01, -3.52700266e+00,  2.50373613e+01],
       [ 6.50084721e-01, -3.07445490e+00,  1.93332108e+01],
       [ 1.97920366e+00, -2.19033073e+00,  1.64740795e+01],
       [ 9.98733127e-01, -1.17642055e+00,  7.59221007e+00],
       [ 1.40297818e+00, -3.25978517e-01,  6.00381113e+00],
       [-2.45146776e+00, -3.43522104e+00,  2.77287940e+01],
       [-7.17993116e-01, -2.42335876e+00,  2.43549554e+01],
       [ 1.69565775e+00, -2.74158163e+00,  2.27244838e+01],
       [ 2.54705998e+00, -1.52380638e+00,  1.92477826e+01],
       [ 2.31195869e+00, -9.39926832e-01